In [1]:
import torch
import torchvision
import numpy as np
!pip install onnx onnxruntime
import onnx
import onnxruntime
from PIL import Image, ImageDraw

In [2]:
#!pip uninstall onnx2torch
#!pip install onnx2torch
!pip install -e git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch

Obtaining onnx2torch from git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch
  Updating ./src/onnx2torch clone
  Running command git fetch -q --tags
  Running command git reset --hard -q ad46e9224f77ccaa06e5a327bc7269aac0778aba
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for onnx2torch (pyproject.toml) ... done
  Created wheel for onnx2torch: filename=onnx2torch-1.5.14-0.editable-py3-none-any.whl size=14406 sha256=eddf2d74c6e6b80fc4546af3b9e9f7a62b9abccba3ff00d2be9da91cd92c0a9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-6tctie5i/wheels/f9/d1/a3/b82562ab7b0402505d5ec31e97cef459ba9bb1251fcc238b54
Successfully built onnx2torch
  Attempting uninstall: onnx2torch
    Found existing installation: onnx2torch 1.5.14
    Uninstalling onnx2torch-1.5.14:
      Successfully uninstalled onnx2

In [ ]:
# Загрузка предобученной модели mask RCNN из torchvision
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Создание входных данных для модели
dummy_input = {'image': torch.rand(2, 3, 300, 400)}

# Запуск модели и вывод результата
output = model(dummy_input['image'])
print(output)

In [5]:
# Конвертация модели в ONNX формат
torch.onnx.export(model, dummy_input['image'], "mask_rcnn.onnx", opset_version=13)

# Загрузка ONNX модели
# onnx_model = onnxruntime.InferenceSession("mask_rcnn.onnx")
onnx_model = onnx.load("mask_rcnn.onnx")

# # Запуск ONNX модели и вывод результата
# input_name = onnx_model.get_inputs()[0].name
# output_name = onnx_model.get_outputs()[0].name
# onnx_output = onnx_model.run([output_name], {input_name: dummy_input['image'].numpy()})
# print(onnx_output)
ort_session = onnxruntime.InferenceSession("mask_rcnn.onnx")

[{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>), 'labels': tensor([], dtype=torch.int64), 'scores': tensor([], grad_fn=<IndexBackward0>), 'masks': tensor([], size=(0, 1, 300, 400))}, {'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>), 'labels': tensor([], dtype=torch.int64), 'scores': tensor([], grad_fn=<IndexBackward0>), 'masks': tensor([], size=(0, 1, 300, 400))}]


/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:5858: UserWarning: Exporting aten::index operator of advanced indexing in opset 13 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


In [6]:
from onnx2torch.converter import convert
torch_model = convert("mask_rcnn.onnx")

NotImplementedError: Converter is not implemented (OperationDescription(domain='', operation_type='If', version=13))